# Smithy API を Bedrock AgentCore Gateway を使用して MCP ツールに変換する

## 概要
Bedrock AgentCore Gateway は、お客様が既存の Smithy API をインフラやホスティングを管理することなく、完全に管理された MCP サーバーに変換する方法を提供します。お客様は Smithy 仕様を持ち込み、それらを MCP ツールに変換できます。このチュートリアルでは、Amazon S3 の Smithy モデルから MCP ツールを作成する方法を示します。エージェントはその後、Amazon S3 に対してクエリを実行し、それに関連する質問に回答できるようになります。

Gateway ワークフローには、エージェントを外部ツールに接続するための以下のステップが含まれます：
* **Gateway 用のツールを作成する** - Smithy 仕様を使用してツールを定義します。
* **Gateway エンドポイントを作成する** - インバウンド認証を備えた MCP エントリポイントとして機能する Gateway を作成します。
* **Gateway にターゲットを追加する** - Gateway が特定のツールにリクエストをルーティングする方法を定義する Smithy ターゲットを設定します。Smithy ファイルの一部であるすべての操作は MCP 互換のツールになり、Gateway エンドポイント URL を通じて利用可能になります。Smithy を介して Amazon S3 API を呼び出すための AWS IAM を使用してアウトバウンド認証を設定します。
* **エージェントコードを更新する** - 統合された MCP インターフェースを通じて設定されたすべてのツールにアクセスするために、エージェントを Gateway エンドポイントに接続します。

![仕組み](images/smithy-apis-gateway.png)

### チュートリアルの詳細


| 情報                 | 詳細                                                      |
|:---------------------|:----------------------------------------------------------|
| チュートリアルタイプ | インタラクティブ                                          |
| AgentCore コンポーネント | AgentCore Gateway, AgentCore Identity                     |
| エージェントフレームワーク | Strands Agents                                            |
| Gateway ターゲットタイプ | Smithy                                                   |
| エージェント         | インフラストラクチャエージェント                          |
| インバウンド認証 IdP | Amazon Cognito                                            |
| アウトバウンド認証   | IAM                                                       |
| LLM モデル           | Anthropic Claude Sonnet 3.7, Amazon Nova Pro              |
| チュートリアルコンポーネント | AgentCore Gateway の作成と呼び出し                       |
| チュートリアル分野   | クロスバーティカル                                        |
| 例の複雑さ           | 簡単                                                      |
| 使用する SDK         | boto3                                                     |

チュートリアルの最初の部分では、いくつかの AmazonCore Gateway ターゲットを作成します。

### チュートリアルのアーキテクチャ
このチュートリアルでは、Amazon S3 の Smithy 仕様で定義された操作を MCP ツールに変換し、Bedrock AgentCore Gateway でホストします。これにより、ユーザーは自分の AWS アカウントの Amazon S3 に関連するあらゆる質問をすることができるようになります。

## 前提条件

このチュートリアルを実行するには以下が必要です：
* Python 3.10+ を搭載した Jupyter notebook
* uv
* AWS 認証情報
* Amazon Cognito

In [ ]:
# 最新の botocore と boto3 ライブラリをダウンロードしていることを確認してください。
import shutil
import subprocess
import sys

def ensure_uv_installed():
    if shutil.which("uv") is None:
        print("🔧 'uv' not found. Installing with pip...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "uv"])
    else:
        print("✅ 'uv' is already installed.")

def uv_install(*packages):
    ensure_uv_installed()
    uv_path = shutil.which("uv")
    print(f"📦 Installing {', '.join(packages)} using uv...")
    subprocess.check_call([uv_path, "pip", "install", *packages])

uv_install("botocore", "boto3")

In [ ]:
import os
# SageMaker ノートブックを使用していない場合は AWS 認証情報を設定する
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

In [ ]:
import os
import sys

# 現在のスクリプトのディレクトリを取得する
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # __file__ が定義されていない場合のフォールバック（例：Jupyter）

# utils.py を含むディレクトリ（一つ上のレベル）に移動する
utils_dir = os.path.abspath(os.path.join(current_dir, '../..'))

# Add to sys.path
sys.path.insert(0, utils_dir)

# 今、 utils をインポートできます
import utils

In [ ]:
#### Gateway が引き受ける IAM ロールを作成する
import utils

agentcore_gateway_iam_role = utils.create_agentcore_gateway_role_s3_smithy("sample-lambdagateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

# Gateway へのインバウンド認証のための Amazon Cognito プールの作成

In [ ]:
# Cognito ユーザープールの作成
import os
import boto3
import requests
import time
from botocore.exceptions import ClientError

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]
scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Creating or retrieving Cognito resources...")
user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {user_pool_id}")

utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("Resource server ensured.")

client_id, client_secret  = utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"Client ID: {client_id}")

# ディスカバリー URL の取得
cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(cognito_discovery_url)

# ゲートウェイの作成

In [ ]:
# CMK なしで Cognito オーソライザーを使用して CreateGateway を行う。前のステップで作成した Cognito ユーザープールを使用する
import boto3
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [client_id],  # クライアントは Cognito で設定された ClientId と一致する必要があります。例： 7rfbikfsm51j2fpaggacgng84g
        "discoveryUrl": cognito_discovery_url
    }
}
create_response = gateway_client.create_gateway(name='DemoS3Smithyv3',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # IAM ロールは Gateway の作成/一覧表示/取得/削除の権限を持っている必要があります
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='AgentCore Gateway with Smithy target'
)
print(create_response)
# GatewayTarget 作成に使用される GatewayID を取得する
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)

# Amazon S3 仕様で Smithy ターゲットを作成する

Amazon S3 API の smithy ゲートウェイターゲットを作成します。Amazon S3 の smithy json ファイルをインポートし、API を MCP ツールに変換します。

#### S3 Smithy API の json ファイルを S3 にアップロードする

In [ ]:
# S3 クライアントを作成する
session = boto3.session.Session()
s3_client = session.client('s3')
sts_client = session.client('sts')

# AWS アカウント ID とリージョンを取得する
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
# Define parameters

パラメータを定義する
bucket_name = '' # OpenAPI json ファイルをアップロードする s3 バケット。
file_path = 'smithy-specs/s3-apis.json'
object_key = 's3-apis.json'
# ファイルを put_object を使用してアップロードし、レスポンスを読み取る
try:
    with open(file_path, 'rb') as file_data:
        response = s3_client.put_object(Bucket=bucket_name, Key=object_key, Body=file_data)

    # アップロードされたオブジェクトの ARN をアカウント ID とリージョンで構築する
    smithy_s3_uri = f's3://{bucket_name}/{object_key}'
    print(f'Uploaded object S3 URI: {smithy_s3_uri}')
except Exception as e:
    print(f'Error uploading file: {e}')

#### ゲートウェイ ターゲットの作成

In [ ]:
# S3 Uri for Smithy 仕様ファイル
smithy_s3_target_config = {
    "mcp": {
          "smithyModel": {
              "s3": {
                  "uri": smithy_s3_uri
              }
          }
      }
}

# IAM 認証情報プロバイダーの設定
credential_config  = {
        "credentialProviderType" : "GATEWAY_IAM_ROLE"
    }

targetname='DemoSmithytargetForS3'
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=targetname,
    description='Smithy Target with S3Uri using SDK',
    targetConfiguration=smithy_s3_target_config,
    credentialProviderConfigurations=[credential_config])

# 不具合を報告するためのリクエスト ID とタイムスタンプを表示します。問題や不具合を報告する際には、これらを含めてください
response_metadata = response['ResponseMetadata']
print(response_metadata)

# Strands エージェントから Bedrock AgentCore Gateway を呼び出す

Strands エージェントは、Model Context Protocol (MCP) 仕様を実装する Bedrock AgentCore Gateway を通じて AWS ツールとシームレスに統合されています。この統合により、AI エージェントと AWS サービス間の安全で標準化された通信が可能になります。

その核心部分として、Bedrock AgentCore Gateway は基本的な MCP API である ListTools と InvokeTools を公開するプロトコル準拠のゲートウェイとして機能します。これらの API により、MCP 準拠のクライアントや SDK は、安全で標準化された方法で利用可能なツールを発見し、対話することができます。Strands エージェントが AWS サービスにアクセスする必要がある場合、これらの MCP 標準化されたエンドポイントを使用してゲートウェイと通信します。

ゲートウェイの実装は (MCP 認証仕様)[https://modelcontextprotocol.org/specification/draft/basic/authorization] に厳密に準拠しており、堅牢なセキュリティとアクセス制御を確保しています。これは、Strands エージェントによるすべてのツール呼び出しが認証ステップを経由することを意味し、強力な機能を提供しながらセキュリティを維持します。

例えば、Strands エージェントが MCP ツールにアクセスする必要がある場合、まず ListTools を呼び出して利用可能なツールを発見し、次に InvokeTools を使用して特定のアクションを実行します。ゲートウェイは必要なすべてのセキュリティ検証、プロトコル変換、サービス連携を処理し、プロセス全体をシームレスかつ安全にします。

このアーキテクチャアプローチにより、MCP 仕様を実装するあらゆるクライアントや SDK がゲートウェイを通じて AWS サービスと対話できるようになり、AI エージェント統合のための汎用性と将来性のあるソリューションとなっています。

In [ ]:
uv_install("mcp[cli]", "strands-agents")

# インバウンド認証のために Amazon Cognito からアクセストークンをリクエストする

In [ ]:
print("Requesting the access token from Amazon Cognito authorizer...May fail for some time till the domain name propogation completes")
token_response = utils.get_token(user_pool_id, client_id, client_secret,scopeString,REGION)
token = token_response["access_token"]
print("Token response:", token)

# AWS アカウントにある s3 リソースについて IT エージェントに質問する

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

## ~/.aws/credentials に設定された IAM グループ/ユーザーは Bedrock モデルへのアクセス権を持っている必要があります
yourmodel = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [ ]:
from strands import Agent
import logging


# ルートの strands ロガーを設定します。問題をデバッグしている場合は、DEBUG に変更してください。
logging.getLogger("strands").setLevel(logging.INFO)

# ログを確認するためのハンドラーを追加する
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

with client:
    # Call the listTools 

# listTools を呼び出す
    tools = client.list_tools_sync()
    # モデルとツールを使用したエージェントの作成
    agent = Agent(model=yourmodel,tools=tools) ## あなたは好きなモデルに置き換えることができます
    #print(f"エージェントに読み込まれたツールは {agent.tool_names} です")

    #print(f"エージェント内のツール設定は {agent.tool_config} です")

    # エージェントをサンプルプロンプトで呼び出します。これは MCP listTools のみを呼び出し、LLM がアクセスできるツールのリストを取得します。以下は実際にはどのツールも呼び出しません。
    agent("Hi , can you list all the s3 buckets. Locate the right tool and call the tool to get the answer")    
    #agent("火星の北部の天気はどうですか")

    # エージェントをサンプルプロンプトで呼び出し、ツールを実行して応答を表示する
    # ログを確認するためのハンドラーを追加する0
    # ログを確認するためのハンドラーを追加する1
    result = client.call_tool_sync(
        tool_use_id="get-insight-weather-1", # ログを確認するためのハンドラーを追加する2
        name=targetname+"___ListBuckets", # ログを確認するためのハンドラーを追加する3
        # ログを確認するためのハンドラーを追加する4
    )
    # ログを確認するためのハンドラーを追加する5
    print(f"Tool Call result: {result['content'][0]['text']}")


# クリーンアップ
IAM ロール、IAM ポリシー、認証情報プロバイダー、AWS Lambda 関数、Cognito ユーザープール、S3 バケットなどの追加リソースも作成されており、クリーンアップの一環として手動で削除する必要があるかもしれません。これは実行する例によって異なります。

## ゲートウェイの削除 (オプション)

In [ ]:
import utils
utils.delete_gateway(gateway_client,gatewayID)